In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
x_train = pd.read_csv(r'D:\Datasets\russian house price\EDA\train_data.csv')
y_train = pd.read_csv(r'D:\Datasets\russian house price\EDA\ytrain_data.csv')
x_test = pd.read_csv(r'D:\Datasets\russian house price\EDA\test_data.csv')
y_test = pd.read_csv(r'D:\Datasets\russian house price\EDA\ytest_data.csv')

In [4]:
x_test.head()

,Unnamed: 0,full_sq,floor,build_year,num_room,kitch_sq,state,product_type,raion_popul,indust_part,...,cafe_sum_1000_min_price_avg,cafe_count_1000_price_high,cafe_sum_1500_min_price_avg,green_part_2000,cafe_sum_2000_min_price_avg,mosque_count_3000,prom_part_5000,cafe_sum_5000_min_price_avg,mosque_count_5000,year
0,3400,40,11.0,1979.0,2.0,6.0,2.0,1,9.990536,0.075779,...,6.214608,0,6.214608,3.837946,6.214608,0,1.517323,6.494570,0,2012
1,5049,32,4.0,1979.0,2.0,6.0,2.0,0,11.628225,0.282798,...,5.703782,0,6.299758,3.726416,6.396930,0,2.801541,6.499712,1,2012
2,23675,41,12.0,2007.0,1.0,10.0,4.0,0,11.186239,0.158249,...,6.907755,0,6.620073,3.476923,6.620073,0,1.724551,6.578181,0,2014
3,29837,84,9.0,2014.0,3.0,10.0,1.0,0,11.352980,0.161532,...,6.506128,0,5.703782,2.910719,6.173786,0,1.994700,6.228787,0,2015
4,18588,46,4.0,1962.0,2.0,5.0,2.0,0,11.019268,0.000000,...,6.109248,0,6.595781,3.018472,6.579251,1,2.296567,6.616253,1,2014


In [5]:
x_train.drop('Unnamed: 0',axis=1,inplace=True)
x_test.drop('Unnamed: 0',axis=1,inplace=True)
y_train.drop('Unnamed: 0',axis=1,inplace=True)
y_test.drop('Unnamed: 0',axis=1,inplace=True)

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
scaler = StandardScaler()
train_data = scaler.fit_transform(x_train)
test_data = scaler.fit_transform(x_test)

In [8]:
train = pd.DataFrame(train_data,columns=x_train.columns)
test = pd.DataFrame(test_data,columns=x_test.columns)

In [9]:
train.shape

(21329, 50)

In [10]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

C:\Users\user\AppData\Local\Temp/ipykernel_16744/2660331953.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [11]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers',2,20)):
        model.add(layers.Dense(units= hp.Int('units_'+str(i),
                                        min_value=32,
                                        max_value =512,
                                        step=32),
                                        activation='relu'))
        model.add(layers.Dense(1,activation='linear'))
        model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',[1e-2,1e-3,1e-4,1e-5])),
        loss ='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [12]:
tuner = RandomSearch(build_model,objective='val_mean_absolute_error',max_trials=5,executions_per_trial=3,
directory='project1.1',project_name='Russiann Data Price')

INFO:tensorflow:Reloading Oracle from existing project project1.1\Russiann Data Price\oracle.json
INFO:tensorflow:Reloading Tuner from project1.1\Russiann Data Price\tuner0.json


In [13]:
tuner.search_space_summary

<bound method BaseTuner.search_space_summary of <keras_tuner.tuners.randomsearch.RandomSearch object at 0x000001FC765C01C0>>

In [14]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import ElasticNet

In [15]:
tuner.search(train,y_train,epochs=10,validation_data = (test,y_test))

INFO:tensorflow:Oracle triggered exit


In [16]:
tuner.results_summary()

Results summary
Results in project1.1\Russiann Data Price
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 8
units_0: 416
learning_rate: 0.01
units_1: 480
units_2: 64
units_3: 128
units_4: 416
units_5: 64
units_6: 192
units_7: 512
units_8: 384
units_9: 288
units_10: 96
units_11: 448
units_12: 288
units_13: 256
units_14: 384
Score: 0.28480525811513263
Trial summary
Hyperparameters:
num_layers: 15
units_0: 320
learning_rate: 0.0001
units_1: 416
units_2: 224
units_3: 480
units_4: 64
units_5: 256
units_6: 64
units_7: 160
units_8: 384
units_9: 416
units_10: 256
units_11: 448
units_12: 448
units_13: 64
units_14: 32
Score: 0.2904561261336009
Trial summary
Hyperparameters:
num_layers: 14
units_0: 64
learning_rate: 0.01
units_1: 192
units_2: 288
units_3: 96
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
Score: 0.30203012625376385
Trial summary


In [17]:
from tensorflow.keras.layers import Dense, Activation
import tensorflow

In [23]:
callback = tensorflow.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=2,
    verbose=1,
    mode="min",
    baseline=None,
    restore_best_weights=False,
)

In [24]:
model = keras.Sequential()
model.add(Dense(units=416,activation='relu',input_dim=50,kernel_initializer='he_uniform'))
model.add(Dense(units=480,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=64,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=128,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=416,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=64,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=192,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=512,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=384,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=288,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=96,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=448,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=288,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=256,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=384,activation='relu',kernel_initializer='he_uniform'))
model.add(layers.Dense(1,activation='linear'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate= 0.01),
        loss ='mean_absolute_error',
        metrics=['mean_absolute_error'])

In [32]:
model_history = model.fit(train,y_train,batch_size = 200,epochs=100,validation_data=(test,y_test),callbacks=[callback])

Epoch 1/100
107/107 [==============================] - 3s 28ms/step - loss: 0.4311 - mean_absolute_error: 0.4311 - val_loss: 0.5338 - val_mean_absolute_error: 0.5338
Epoch 2/100
107/107 [==============================] - 2s 23ms/step - loss: 0.4075 - mean_absolute_error: 0.4075 - val_loss: 0.5978 - val_mean_absolute_error: 0.5978
Epoch 3/100
107/107 [==============================] - 2s 20ms/step - loss: 0.3848 - mean_absolute_error: 0.3848 - val_loss: 0.3264 - val_mean_absolute_error: 0.3264
Epoch 4/100
107/107 [==============================] - 2s 20ms/step - loss: 0.4224 - mean_absolute_error: 0.4224 - val_loss: 0.4074 - val_mean_absolute_error: 0.4074
Epoch 5/100
107/107 [==============================] - 2s 20ms/step - loss: 0.3413 - mean_absolute_error: 0.3413 - val_loss: 0.3174 - val_mean_absolute_error: 0.3174
Epoch 6/100
107/107 [==============================] - 2s 21ms/step - loss: 0.3528 - mean_absolute_error: 0.3528 - val_loss: 0.3014 - val_mean_absolute_error: 0.3014
Epoc

In [33]:
y_pred = model.predict(test)

In [34]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_test,y_pred))

0.3237588585372859


In [35]:
model.save("House_price.h5")